In [24]:
from pathlib import Path
import os
import sys
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
    
import warnings
from cycler import cycler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold

from settings.paths import match_path, validation_path, rf_path, bnn_path, flex_path
from settings.columns import specz, calculate_colors, create_colors, list_feat
from utils.metrics import nmad, bias ,out_frac, rmse, print_metrics_xval, print_metrics_test
from utils.preprocessing import create_bins, rename_aper, mag_redshift_selection, prep_wise, missing_input, flag_observation
from utils.crossvalidation import xval_results, save_folds


plt.rcParams["font.size"] = 22
blue = (0, 0.48, 0.70)
orange = (230/255,159/255, 0)
yellow = (0.94, 0.89, 0.26)
pink = (0.8, 0.47, 0.65)
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['axes.prop_cycle'] = cycler('color', CB_color_cycle)

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Read results

In [20]:
# Random Forest
rf_all = pd.read_csv(os.path.join(rf_path,"test_z_broad+GALEX+WISE+narrow+flags.csv"))
rf_broad = pd.read_csv(os.path.join(rf_path,"test_z_broad+GALEX+WISE+flags.csv"))

# BMDN
bmdn_all = pd.read_csv(os.path.join(bnn_path,"bnn_BNWG_flag.csv"))
bmdn_broad = pd.read_csv(os.path.join(bnn_path,"bnn_BWG_flag.csv"))

# FlexCoDE
flex_all = pd.read_csv(os.path.join(flex_path,"test_z_broad+GALEX+WISE+narrow+flags.csv"))
flex_broad = pd.read_csv(os.path.join(flex_path,"test_z_broad+GALEX+WISE+flags.csv"))

# Test set
test = pd.read_csv(os.path.join(validation_path,"test.csv"), index_col="index")

In [13]:
print_metrics(rf_all, xval=False)

RMSE 0.409 0.409
NMAD 0.0889 0.0889
bias 0.0021 0.0021
n15 0.2186 0.2186
n30 0.0649 0.0649


In [14]:
print_metrics(rf_broad, xval=False)

RMSE 0.4227 0.4227
NMAD 0.1003 0.1003
bias -0.0024 -0.0024
n15 0.2245 0.2245
n30 0.0683 0.0683


In [25]:
print_metrics_test(test.Z, flex_all.z_flex_peak)

RMSE 0.4955 0.4955
NMAD nan nan
bias 0.0251 0.0251
n15 0.162 0.162
n30 0.0985 0.0985
